# Pipeline de Entrenamiento con Feature Views de Hopsworks

Este notebook implementa un pipeline completo para:
1. Conectar al feature store de Hopsworks
2. Crear una feature view con características seleccionadas
3. Procesar los datos para crear variables de rezago (lags) y target
4. Preparar los datos para entrenamiento de modelos

In [9]:
# 1. Configuración inicial
from datetime import datetime
from src import config
import hopsworks
import pandas as pd
import logging

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('feature_view_creation')

In [10]:
# 2. Conectar a Hopsworks y al Feature Store
try:
    # Login y conexión al proyecto
    project = hopsworks.login(
        api_key_value=config.HOPSWORKS_API_KEY, 
        project=config.HOPSWORKS_PROJECT_NAME)
    
    # Conexión al feature store
    feature_store = project.get_feature_store()
    
    # Conexión al feature group
    feature_group = feature_store.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=config.FEATURE_GROUP_VERSION
    )
    
    logger.info(f"Conexión exitosa al Feature Group: {feature_group.name} (v{feature_group.version})")
    
except Exception as e:
    logger.error(f"Error en conexión: {e}")
    raise

2025-08-16 10:13:22,115 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-16 10:13:22,118 INFO: Initializing external client
2025-08-16 10:13:22,119 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-08-16 10:13:22,118 INFO: Initializing external client
2025-08-16 10:13:22,119 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-16 10:13:23,254 INFO: Python Engine initialized.
2025-08-16 10:13:23,254 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
2025-08-16 10:13:24,931 INFO: Conexión exitosa al Feature Group: times_series_bolleria_feature_group (v1)
2025-08-16 10:13:24,931 INFO: Conexión exitosa al Feature Group: times_series_bolleria_feature_group (v1)


In [11]:
# 3. Crear/obtener feature view con características seleccionadas
try:
    # Características específicas a incluir
    selected_features = ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']
    feature_view_name = config.FEATURE_VIEW_NAME
    feature_view_version = 1
    
    # Intentar obtener la feature view existente primero
    try:
        feature_view = feature_store.get_feature_view(
            name=feature_view_name,
            version=feature_view_version
        )
        logger.info(f"Feature view existente recuperada: {feature_view.name} (v{feature_view.version})")
    
    except:
        # Si no existe, crear una nueva
        # Obtener objetos Feature para las características seleccionadas
        selected_feature_objects = [f for f in feature_group.features if f.name in selected_features]
        
        # Crear query con características seleccionadas
        specific_query = feature_group.select(selected_feature_objects)
        
        # Crear la feature view
        feature_view = feature_store.create_feature_view(
            name=feature_view_name,
            version=feature_view_version,
            query=specific_query,
            description=f"Feature view con características: {', '.join(selected_features)}"
        )
        logger.info(f"Nueva feature view creada: {feature_view.name} (v{feature_view.version})")
    
except Exception as e:
    logger.error(f"Error al crear/obtener feature view: {e}")
    raise

2025-08-16 10:13:33,198 INFO: Feature view existente recuperada: times_series_bolleria_feature_view (v1)


In [12]:
# 4. Obtener datos de la feature view
try:
    # Obtener datos en batch normal
    data = feature_view.get_batch_data()
    
    # Mostrar resumen de los datos obtenidos
    logger.info(f"Datos obtenidos: {data.shape[0]} filas, {data.shape[1]} columnas")
    logger.info(f"Columnas disponibles: {list(data.columns)}")
    print("Muestra de datos:")
    print(data.head(3))
    
except Exception as e:
    logger.error(f"Error al obtener datos: {e}")
    raise

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.49s) 
2025-08-16 10:13:47,676 INFO: Datos obtenidos: 132 filas, 5 columnas
2025-08-16 10:13:47,678 INFO: Columnas disponibles: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']
Muestra de datos:
    familia  base_imponible  is_summer_peak  is_easter  \
0  BOLLERIA          641.56               0          0   
1  BOLLERIA          725.72               0          0   
2  BOLLERIA          950.70               0          0   

                 week_start  
0 2023-02-06 00:00:00+00:00  
1 2025-02-24 00:00:00+00:00  
2 2023-09-18 00:00:00+00:00  
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.49s) 
2025-08-16 10:13:47,676 INFO: Datos obtenidos: 132 filas, 5 columnas
2025-08-16 10:13:47,678 INFO: Columnas disponibles: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']
Muestra de datos:
    familia  base_imponible  is_summer_peak  is

In [13]:
# 5. Obtener datos de entrenamiento (training_data)
try:
    # Obtener datos de entrenamiento - devuelve una tupla (X, y)
    # Nota: y puede ser None si no hay target definido en la feature view
    df_ts = feature_view.training_data()
    
    # Verificar componentes
    if isinstance(df_ts, tuple) and len(df_ts) >= 1:
        X_features = df_ts[0]
        logger.info(f"Datos de características obtenidos: {X_features.shape[0]} filas, {X_features.shape[1]} columnas")
        print("Muestra de características:")
        print(X_features.head(3))
    else:
        logger.error("Formato inesperado de datos de entrenamiento")
        raise ValueError("El formato de datos no es el esperado")
    
except Exception as e:
    logger.error(f"Error al obtener datos de entrenamiento: {e}")
    raise

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
2025-08-16 10:14:07,452 INFO: Datos de características obtenidos: 132 filas, 5 columnas
Muestra de características:
    familia  base_imponible  is_summer_peak  is_easter  \
0  BOLLERIA          641.56               0          0   
1  BOLLERIA          725.72               0          0   
2  BOLLERIA          950.70               0          0   

                  week_start  
0  2023-02-06 00:00:00+00:00  
1  2025-02-24 00:00:00+00:00  
2  2023-09-18 00:00:00+00:00  
2025-08-16 10:14:07,452 INFO: Datos de características obtenidos: 132 filas, 5 columnas
Muestra de características:
    familia  base_imponible  is_summer_peak  is_easter  \
0  BOLLERIA          641.56               0          0   
1  BOLLERIA          725.72               0          0   
2  BOLLERIA          950.70               0          0   

      

In [14]:
# 6. Procesar datos para entrenamiento
from src.data_utils import transformar_features_target

try:
    # Procesar datos usando la función mejorada que acepta tuplas directamente
    features_and_target = transformar_features_target(
        df_ts,
        lags_list=[1, 2, 3, 52], 
        columna_target='base_imponible',
        cols_exogenas=None,
        periodos_adelante=1,
        eliminar_nulos=True,
        return_format='dataframe'  # Obtenemos un único DataFrame con features y target
    )
    
    # Mostrar información de los datos procesados
    logger.info(f"Datos procesados: {features_and_target.shape[0]} filas, {features_and_target.shape[1]} columnas")
    logger.info(f"Variables disponibles: {list(features_and_target.columns)}")
    print("\nMuestra de datos procesados:")
    print(features_and_target.head(3))
    
    # Guardar para uso posterior (opcional)
    processed_path = "data/processed/bolleria_features_target.parquet"
    features_and_target.to_parquet(processed_path)
    logger.info(f"Datos procesados guardados en: {processed_path}")
    
except Exception as e:
    logger.error(f"Error al procesar datos: {e}")
    raise

2025-08-16 10:18:40,108 INFO: Detectada entrada tipo tupla con 2 elementos
2025-08-16 10:18:40,109 INFO: Usando el primer elemento de la tupla como DataFrame: (132, 5)
2025-08-16 10:18:40,112 INFO: Retornando DataFrame combinado: (79, 6)
2025-08-16 10:18:40,113 INFO: Datos procesados: 79 filas, 6 columnas
2025-08-16 10:18:40,113 INFO: Variables disponibles: ['base_imponible_lag1', 'base_imponible_lag2', 'base_imponible_lag3', 'base_imponible_lag52', 'week_start', 'target']

Muestra de datos procesados:
     base_imponible_lag1  base_imponible_lag2  base_imponible_lag3  \
41                572.51               534.79               563.18   
72                597.65               572.51               534.79   
114               680.30               597.65               572.51   

     base_imponible_lag52                 week_start  target  
41                 825.11  2024-01-15 00:00:00+00:00  680.30  
72                 658.40  2024-01-22 00:00:00+00:00  603.99  
114                741

In [ ]:
# 7. Entrenar modelo básico con las características seleccionadas
try:
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error, r2_score
    import numpy as np
    
    # Definir características y objetivo (ajustar según datos)
    X = data[['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']]
    y = data['ventas'] if 'ventas' in data.columns else data.iloc[:, -1]  # Última columna como fallback
    
    # División train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenar modelo
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluar
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    # Resultados
    logger.info(f"Resultados del modelo RandomForest:")
    logger.info(f"RMSE: {rmse:.4f}")
    logger.info(f"R²: {r2:.4f}")
    
    # Importancia de características
    importances = pd.DataFrame({
        'Característica': X.columns,
        'Importancia': model.feature_importances_
    }).sort_values('Importancia', ascending=False)
    
    print("Importancia de características:")
    print(importances)
    
except Exception as e:
    logger.error(f"Error en entrenamiento: {e}")